# Notebook 02: Exploration des Stratégies de Récupération (RAG)

Ce notebook explore les capacités de notre `RetrievalEngine` (moteur de récupération) pour extraire des informations pertinentes à partir de la base de connaissances MongoDB que nous avons constituée. Nous allons tester différentes stratégies, notamment :
*   Recherche vectorielle simple (basée sur la similarité sémantique).
*   Recherche vectorielle avec filtres sur les métadonnées.
*   (Optionnel, si implémenté dans `RetrievalEngine`) Recherche hybride combinant recherche vectorielle et recherche textuelle par mots-clés.

**Prérequis :**

1.  **Environnement Configuré** : Avoir exécuté le notebook `00_setup_environment.ipynb` et s'assurer que le fichier `.env` à la racine du projet est correctement rempli (clés API, `MONGODB_URI`, `OLLAMA_BASE_URL` si applicable, etc.).
2.  **Base de Données Populée** : Avoir exécuté le notebook `01_data_ingestion_and_embedding.ipynb` (ou le script `scripts/run_ingestion.py`). Une collection MongoDB (par exemple, `arxiv_chunks_notebook_test_ollama` ou le nom que vous avez utilisé) doit exister et contenir des documents "chunkés" avec leurs embeddings.
3.  **Configuration de l'Embedding pour les Requêtes** :
    *   Le `RetrievalEngine` vectorise les requêtes de recherche en utilisant le fournisseur d'embedding défini par `DEFAULT_EMBEDDING_PROVIDER` dans `config/settings.py` (influencé par `.env`).
    *   **Si `DEFAULT_EMBEDDING_PROVIDER="ollama"`** (par défaut) :
        *   Assurez-vous que `OLLAMA_BASE_URL` est correct dans `.env`.
        *   Le modèle d'embedding spécifié par `OLLAMA_EMBEDDING_MODEL_NAME` (ex: `nomic-embed-text`) doit être disponible sur votre instance Ollama (`ollama pull nomic-embed-text`).
    *   **Si `DEFAULT_EMBEDDING_PROVIDER="openai"`** :
        *   `OPENAI_API_KEY` doit être configurée dans `.env`.
    *   **Si `DEFAULT_EMBEDDING_PROVIDER="huggingface"`** :
        *   Aucune clé API spécifique n'est généralement requise pour les modèles Sentence Transformers locaux utilisés via LangChain/LlamaIndex.
4.  **Instance MongoDB Accessible** : Votre serveur MongoDB (local ou Atlas) doit être en cours d'exécution et accessible via le `MONGODB_URI` de votre fichier `.env`.

In [1]:
# --- Imports Standards et de Configuration ---
import logging
import sys
from pathlib import Path
import os
import json # Pour un affichage formaté des métadonnées
from typing import Optional, List, Dict, Any # Ajout de Dict et Any pour type hinting

# --- Configuration du Projet et Logging ---
# Assurer que la racine du projet est dans sys.path pour les imports de src/ et config/
project_root_path = Path.cwd().parent 
if str(project_root_path) not in sys.path:
    sys.path.append(str(project_root_path))
    print(f"Ajout de {project_root_path} à sys.path")

from dotenv import load_dotenv
dotenv_path = project_root_path / ".env"
if dotenv_path.exists():
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Variables d'environnement chargées depuis : {dotenv_path}")
else:
    print(f"ATTENTION: Fichier .env non trouvé à {dotenv_path}. Assurez-vous qu'il est à la racine du projet (makers/).")
    print("Les configurations (clés API, URI MongoDB, etc.) pourraient être manquantes.")

from config.settings import settings
from config.logging_config import setup_logging
# Import du RetrievalEngine et de la structure de données RetrievedNode
from src.rag.retrieval_engine import RetrievalEngine, RetrievedNode 
# MongoDBManager pourrait être utile pour des vérifications directes (optionnel ici)
# from src.vector_store.mongodb_manager import MongoDBManager 

# --- Configuration du Logging pour ce Notebook ---
setup_logging(level="INFO") # Changer à "DEBUG" pour des logs plus détaillés de RetrievalEngine
logger = logging.getLogger("nb_02_rag_exploration")

logger.info("--- Initialisation du Notebook 02: Exploration des Stratégies RAG ---")

# --- Vérification des Prérequis pour l'Embedding et MongoDB (Crucial pour RetrievalEngine) ---
active_embedding_provider = settings.DEFAULT_EMBEDDING_PROVIDER.lower()
logger.info(f"Ce notebook utilisera le fournisseur d'embedding configuré : '{active_embedding_provider}' pour RetrievalEngine.")

error_messages = []
if active_embedding_provider == "openai":
    if not settings.OPENAI_API_KEY:
        error_messages.append("ERREUR : DEFAULT_EMBEDDING_PROVIDER='openai', mais OPENAI_API_KEY manque dans .env.")
elif active_embedding_provider == "ollama":
    if not settings.OLLAMA_BASE_URL:
        error_messages.append("ERREUR : DEFAULT_EMBEDDING_PROVIDER='ollama', mais OLLAMA_BASE_URL manque dans .env.")
    if not settings.OLLAMA_EMBEDDING_MODEL_NAME:
         error_messages.append("ERREUR : DEFAULT_EMBEDDING_PROVIDER='ollama', mais OLLAMA_EMBEDDING_MODEL_NAME manque dans .env/settings.")
    logger.info(f"  Pour Ollama, assurez-vous que le serveur est accessible à {settings.OLLAMA_BASE_URL} et que le modèle '{settings.OLLAMA_EMBEDDING_MODEL_NAME}' est disponible.")
# Pas de vérification de clé pour "huggingface" (Sentence Transformers locaux)

if not settings.MONGODB_URI:
    error_messages.append("ERREUR CRITIQUE : MONGODB_URI non trouvé dans .env. RetrievalEngine ne pourra pas se connecter.")

if error_messages:
    for err in error_messages:
        logger.error(err)
    logger.error("Veuillez corriger les erreurs de configuration ci-dessus avant de continuer.")
    # Vous pourriez vouloir arrêter l'exécution ici si des erreurs critiques sont détectées.
    # raise RuntimeError("Configuration incomplète pour RetrievalEngine. Voir logs.")
else:
    logger.info("Vérification initiale des configurations (clés API, URI) : OK.")


# --- Configuration des Noms de Collection et d'Index MongoDB pour ce Test ---
# Par défaut, utiliser la collection et les index créés par le notebook 01.
# Si le notebook 01 a utilisé un nom de collection dépendant du provider, assurez-vous que c'est le même ici.
# Exemple de nom de collection du notebook 01 : "arxiv_chunks_notebook_test_ollama"
COLLECTION_NAME_FOR_RAG_TEST = f"arxiv_chunks_notebook_test_{active_embedding_provider}"
VECTOR_INDEX_NAME_FOR_RAG_TEST = "vector_index_notebook_test" # Doit correspondre à ce qui a été créé dans le notebook 01
TEXT_INDEX_NAME_FOR_RAG_TEST = "text_index_notebook_test" # Doit correspondre

# Alternative: Utiliser les noms par défaut du MongoDBManager si vous avez ingéré des données avec run_ingestion.py
# from src.vector_store.mongodb_manager import MongoDBManager # Déjà importé plus haut si besoin
# COLLECTION_NAME_FOR_RAG_TEST = MongoDBManager.DEFAULT_CHUNK_COLLECTION_NAME
# VECTOR_INDEX_NAME_FOR_RAG_TEST = MongoDBManager.DEFAULT_VECTOR_INDEX_NAME
# TEXT_INDEX_NAME_FOR_RAG_TEST = MongoDBManager.DEFAULT_TEXT_INDEX_NAME

logger.info(f"Le RetrievalEngine ciblera la collection MongoDB : '{COLLECTION_NAME_FOR_RAG_TEST}'")
logger.info(f"Index vectoriel à utiliser : '{VECTOR_INDEX_NAME_FOR_RAG_TEST}'")
logger.info(f"Index textuel (pour recherche hybride si applicable) : '{TEXT_INDEX_NAME_FOR_RAG_TEST}'")

logger.info("Configuration initiale du notebook terminée.")

Variables d'environnement chargées depuis : /home/facetoface/makers/.env
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO - --- Initialisation du Notebook 02: Exploration des Stratégies RAG ---
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO - Ce notebook utilisera le fournisseur d'embedding configuré : 'ollama' pour RetrievalEngine.
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO -   Pour Ollama, assurez-vous que le serveur est accessible à http://localhost:11434 et que le modèle 'nomic-embed-text' est disponible.
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO - Vérification initiale des configurations (clés API, URI) : OK.
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO - Le RetrievalEngine ciblera la collection MongoDB : 'arxiv_chunks_notebook_test_ollama'
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO - Index vectoriel à utiliser : 'vector_index_notebook_test'
2025-06-03 23:28:46 - nb_02_rag_exploration - INFO - Index textuel (pour recherche hybride si applicable

### Initialisation du `RetrievalEngine`

Nous allons maintenant créer une instance de notre `RetrievalEngine`. 
Lors de son initialisation, le `RetrievalEngine` (basé sur LlamaIndex) :
1.  Configure le modèle d'embedding global en fonction du `DEFAULT_EMBEDDING_PROVIDER`.
2.  Se connecte à MongoDB en utilisant le `MONGODB_URI`.
3.  Initialise un `MongoDBAtlasVectorSearch` (ou un store vectoriel LlamaIndex équivalent) pointant vers la collection et l'index vectoriel spécifiés.
4.  Charge l'index vectoriel (`VectorStoreIndex`) à partir de ce store.
5.  Configure un "retriever" par défaut à partir de cet index.

Assurez-vous que la collection (`COLLECTION_NAME_FOR_RAG_TEST`) et l'index vectoriel (`VECTOR_INDEX_NAME_FOR_RAG_TEST`) existent et contiennent des données avec des embeddings correspondant au fournisseur configuré.

In [2]:
# Initialiser la variable pour l'instance du RetrievalEngine
retrieval_engine_instance: Optional[RetrievalEngine] = None

# S'assurer que les erreurs de configuration précédentes n'empêchent pas une tentative d'initialisation
# (le code précédent logue des erreurs mais n'arrête pas forcément l'exécution)
if error_messages: # error_messages défini dans la cellule précédente
    logger.error("Initialisation du RetrievalEngine annulée en raison d'erreurs de configuration détectées précédemment.")
else:
    logger.info(f"Tentative d'initialisation du RetrievalEngine...")
    try:
        retrieval_engine_instance = RetrievalEngine(
            collection_name=COLLECTION_NAME_FOR_RAG_TEST,
            vector_index_name=VECTOR_INDEX_NAME_FOR_RAG_TEST
            # text_search_index_name peut être ajouté si le moteur le gère pour l'hybride
            # text_key et embedding_key sont généralement pris par défaut par RetrievalEngine (via settings ou LlamaIndex defaults)
        )
        logger.info("RetrievalEngine initialisé avec succès.")
    except Exception as e:
        logger.error(f"ERREUR CRITIQUE lors de l'initialisation du RetrievalEngine: {e}", exc_info=True)
        logger.error("Causes possibles :")
        logger.error("  - URI MongoDB incorrect ou serveur inaccessible.")
        logger.error(f"  - Collection '{COLLECTION_NAME_FOR_RAG_TEST}' ou index vectoriel '{VECTOR_INDEX_NAME_FOR_RAG_TEST}' non trouvés ou mal configurés.")
        logger.error("  - Problème avec le fournisseur d'embedding (ex: clé API OpenAI manquante, serveur Ollama inaccessible, modèle Ollama non trouvé).")
        logger.error("  - Incohérence entre les embeddings dans la DB et le modèle d'embedding actuel.")
        retrieval_engine_instance = None # S'assurer qu'il est None en cas d'échec

# --- Fonction d'Aide pour Afficher les Résultats de Récupération ---
def display_retrieved_nodes(
    retrieved_nodes: Optional[List[RetrievedNode]], 
    query_text: str,
    search_type: str = "Vector Search"
) -> None:
    """Affiche les nœuds récupérés de manière formatée."""
    print(f"\n--- Résultats pour la Requête ({search_type}) : \"{query_text}\" ---")
    if not retrieved_nodes:
        print("Aucun document pertinent n'a été trouvé pour cette requête.")
        print("--------------------------------------------------")
        return

    for i, node in enumerate(retrieved_nodes):
        print(f"\nRésultat # {i + 1}:")
        print(f"  Score de Similarité : {node.score:.4f}" if node.score is not None else "  Score de Similarité : N/A")
        
        # Accès sécurisé aux métadonnées
        metadata = node.metadata if node.metadata else {}
        print(f"  ID du Chunk         : {metadata.get('chunk_id', 'N/A')}") # Si 'chunk_id' est dans les métadonnées
        print(f"  ID ArXiv (source)   : {metadata.get('arxiv_id', 'N/A')}")
        print(f"  Titre du Document   : {metadata.get('original_document_title', 'N/A')}")
        # Afficher d'autres métadonnées si elles existent et sont pertinentes
        # Par exemple, l'année de publication si ajoutée
        # published_year = metadata.get('published_year', metadata.get('published', {}).get('$date', '')[:4] if isinstance(metadata.get('published'), dict) else metadata.get('published', '')[:4])
        # print(f"  Année Publication   : {published_year if published_year else 'N/A'}")

        # Afficher un extrait du texte du chunk
        text_excerpt = node.text[:350].replace('\n', ' ') + "..." if node.text else "N/A"
        print(f"  Extrait du Texte    : {text_excerpt}")
    print("--------------------------------------------------")

# Test rapide pour voir si l'instance est créée
if retrieval_engine_instance:
    logger.info("L'instance RetrievalEngine est prête à être utilisée.")
else:
    logger.error("L'instance RetrievalEngine n'a pas pu être créée. Les étapes suivantes échoueront.")

2025-06-03 23:29:08 - nb_02_rag_exploration - INFO - Tentative d'initialisation du RetrievalEngine...
2025-06-03 23:29:08 - src.rag.retrieval_engine - INFO - Configuring LlamaIndex embed_model for provider: ollama
2025-06-03 23:29:08 - src.rag.retrieval_engine - INFO - Configured Ollama embedding model: nomic-embed-text
2025-06-03 23:29:09 - src.rag.retrieval_engine - INFO - Configured MongoDB vector store for collection: arxiv_chunks_notebook_test_ollama
2025-06-03 23:29:09 - src.rag.retrieval_engine - INFO - Initialized vector store index and retriever
2025-06-03 23:29:09 - src.rag.retrieval_engine - INFO - RetrievalEngine initialized with LlamaIndex components
2025-06-03 23:29:09 - nb_02_rag_exploration - INFO - RetrievalEngine initialisé avec succès.
2025-06-03 23:29:09 - nb_02_rag_exploration - INFO - L'instance RetrievalEngine est prête à être utilisée.


### Définition des Requêtes d'Exemple

Pour tester notre moteur de récupération, nous avons besoin de quelques requêtes. 
Ces requêtes devraient être pertinentes par rapport au contenu du corpus que vous avez ingéré. 
Si vous avez utilisé la requête `ARXIV_QUERY_NOTEBOOK = "explainable artificial intelligence for robotics"` du notebook 01, les exemples ci-dessous devraient être adaptés. Sinon, modifiez-les en fonction de votre corpus.

In [3]:
# Liste de requêtes d'exemple pour tester le RAG
sample_queries_for_rag: List[str] = [
    "What are the main challenges in applying reinforcement learning to robotic manipulation?",
    "Techniques for explainable AI (XAI) in robot decision-making processes.",
    "How can sim-to-real transfer be improved for reinforcement learning agents in robotics?",
    "Common algorithms for path planning in multi-robot systems using RL.",
    "What are the ethical considerations for autonomous robots using advanced AI?",
    "How is Large Language Models (LLMs) applied to robotics?" # Ajout d'une requête plus générale
]

# Sélectionner une requête pour les tests initiaux
# Vous pouvez changer l'index pour tester différentes requêtes
selected_test_query: str = sample_queries_for_rag[0] 

logger.info(f"Requête de test sélectionnée pour les exemples suivants : \"{selected_test_query}\"")

2025-06-03 23:29:26 - nb_02_rag_exploration - INFO - Requête de test sélectionnée pour les exemples suivants : "What are the main challenges in applying reinforcement learning to robotic manipulation?"


### Stratégie 1: Recherche Vectorielle Simple

Cette première stratégie est la forme la plus basique de RAG :
1.  La requête de l'utilisateur (`selected_test_query`) est convertie en un vecteur d'embedding en utilisant le même modèle d'embedding que celui utilisé pour les chunks stockés.
2.  Ce vecteur de requête est ensuite comparé aux vecteurs d'embedding des chunks dans MongoDB pour trouver les plus similaires (distance cosinus généralement).
3.  Les chunks les plus similaires sont retournés.

Nous utilisons la méthode `retrieve_simple_vector_search` (ou un nom similaire, ex: `retrieve`) de notre `RetrievalEngine`.

In [5]:
logger.info(f"\n--- Début Stratégie 1: Recherche Vectorielle Simple ---")
logger.info(f"Exécution de la recherche vectorielle simple pour la requête : \"{selected_test_query}\"")

retrieved_nodes_simple_search: Optional[List[RetrievedNode]] = None

if retrieval_engine_instance:
    try:
        # CORRECTION: Utiliser le nom de méthode correct 'retrieve_simple_vector_search'
        retrieved_nodes_simple_search = retrieval_engine_instance.retrieve_simple_vector_search(
            query_text=selected_test_query,
            top_k=3 # Demander les 3 résultats les plus pertinents
            # metadata_filters est optionnel et non utilisé pour cette recherche simple
        )
        
        # Afficher les résultats en utilisant notre fonction d'aide
        display_retrieved_nodes(
            retrieved_nodes_simple_search, 
            selected_test_query, 
            search_type="Recherche Vectorielle Simple"
        )

    except Exception as e:
        logger.error(f"Erreur lors de la recherche vectorielle simple : {e}", exc_info=True)
        print(f"Une erreur est survenue. Assurez-vous que le RetrievalEngine est initialisé et que la base de données est accessible et peuplée.")
else:
    logger.error("RetrievalEngine non initialisé. Impossible d'exécuter la recherche.")

logger.info(f"--- Fin Stratégie 1 ---")

2025-06-03 23:30:51 - nb_02_rag_exploration - INFO - 
--- Début Stratégie 1: Recherche Vectorielle Simple ---
2025-06-03 23:30:51 - nb_02_rag_exploration - INFO - Exécution de la recherche vectorielle simple pour la requête : "What are the main challenges in applying reinforcement learning to robotic manipulation?"
2025-06-03 23:30:51 - httpx - INFO - HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
2025-06-03 23:30:52 - src.rag.retrieval_engine - INFO - Retrieved 0 nodes for query: 'What are the main challenges in applying reinforce...'

--- Résultats pour la Requête (Recherche Vectorielle Simple) : "What are the main challenges in applying reinforcement learning to robotic manipulation?" ---
Aucun document pertinent n'a été trouvé pour cette requête.
--------------------------------------------------
2025-06-03 23:30:52 - nb_02_rag_exploration - INFO - --- Fin Stratégie 1 ---


### Stratégie 2: Recherche Vectorielle avec Filtres de Métadonnées

Souvent, une simple recherche vectorielle sémantique n'est pas suffisante. Nous pouvons vouloir affiner les résultats en ne considérant que les chunks qui correspondent à certains critères de métadonnées. Par exemple, récupérer des documents :
*   Publiés une année spécifique.
*   Appartenant à une catégorie ArXiv principale particulière.
*   Provenant d'un article (ArXiv ID) spécifique.

Le `RetrievalEngine` (via la méthode `retrieve_simple_vector_search` ou une méthode dédiée si elle existe) devrait permettre de passer de tels filtres. La structure des filtres attendus par LlamaIndex est généralement une liste de dictionnaires ou d'objets `ExactMatchFilter`.

Pour cet exemple, nous allons essayer de filtrer par `arxiv_id` (pour simuler la recherche de chunks d'un document spécifique) et potentiellement par `primary_category`. Assurez-vous que les valeurs de filtre que vous utilisez existent réellement dans les métadonnées de votre corpus.

In [6]:
logger.info(f"\n--- Début Stratégie 2: Recherche Vectorielle avec Filtres de Métadonnées ---")

# Choisir une requête pour ce test (peut être la même que précédemment ou une autre)
query_for_filtered_search = selected_test_query # Ou sample_queries_for_rag[1] par exemple
logger.info(f"Requête pour la recherche filtrée : \"{query_for_filtered_search}\"")

# Définition des filtres de métadonnées
# Le format attendu par retrieve_simple_vector_search est List[Dict[str, Any]]
# où chaque dict est comme {"key": "metadata_field_name", "value": "valeur_a_filtrer"}
#
# Exemple 1: Filtrer par un ID ArXiv spécifique.
# Pour cela, nous devons d'abord connaître un ID ArXiv présent dans nos données.
# Idéalement, on le récupère dynamiquement à partir des résultats précédents ou d'une exploration.
# Supposons que l'un des ArXiv IDs des chunks récupérés à l'étape précédente était '2505.24878' (à adapter).

# Tentative de récupérer un arxiv_id à partir des résultats précédents (si disponibles)
target_arxiv_id_for_filter = None
if retrieved_nodes_simple_search and retrieved_nodes_simple_search[0].metadata:
    target_arxiv_id_for_filter = retrieved_nodes_simple_search[0].metadata.get('arxiv_id')

if not target_arxiv_id_for_filter:
    logger.warning("Impossible de déterminer un target_arxiv_id à partir des résultats précédents pour le filtre. Utilisation d'une valeur par défaut ou test désactivé.")
    # Mettez ici un ID ArXiv que vous savez exister dans votre collection de test si besoin, ex:
    # target_arxiv_id_for_filter = "ID_ARXIV_CONNU_DANS_VOTRE_JEU_DE_TEST" 
    # Ou, si vous ne pouvez pas en fournir un, la recherche filtrée pourrait ne pas être très utile.

if target_arxiv_id_for_filter:
    logger.info(f"Application d'un filtre sur arxiv_id = '{target_arxiv_id_for_filter}'")
    metadata_filters_example = [
        {"key": "arxiv_id", "value": target_arxiv_id_for_filter}
    ]
    
    # Vous pouvez ajouter d'autres filtres, par exemple sur la catégorie principale
    # (assurez-vous que 'primary_category' est bien une clé dans vos métadonnées de chunk)
    # metadata_filters_example.append({"key": "primary_category", "value": "cs.RO"}) # Exemple: Robotics

    retrieved_nodes_filtered_search: Optional[List[RetrievedNode]] = None
    if retrieval_engine_instance:
        try:
            retrieved_nodes_filtered_search = retrieval_engine_instance.retrieve_simple_vector_search(
                query_text=query_for_filtered_search,
                top_k=3, # Demander 3 résultats
                metadata_filters=metadata_filters_example
            )
            
            display_retrieved_nodes(
                retrieved_nodes_filtered_search, 
                query_for_filtered_search, 
                search_type=f"Recherche Filtrée (arxiv_id: {target_arxiv_id_for_filter})"
            )

        except Exception as e:
            logger.error(f"Erreur lors de la recherche vectorielle filtrée : {e}", exc_info=True)
            print(f"Une erreur est survenue lors de la recherche filtrée.")
    else:
        logger.error("RetrievalEngine non initialisé. Impossible d'exécuter la recherche filtrée.")
else:
    logger.warning("Aucun target_arxiv_id n'a été défini pour le filtre. La recherche filtrée par ArXiv ID est sautée.")

logger.info(f"--- Fin Stratégie 2 ---")

2025-06-03 23:31:52 - nb_02_rag_exploration - INFO - 
--- Début Stratégie 2: Recherche Vectorielle avec Filtres de Métadonnées ---
2025-06-03 23:31:52 - nb_02_rag_exploration - INFO - Requête pour la recherche filtrée : "What are the main challenges in applying reinforcement learning to robotic manipulation?"
2025-06-03 23:31:52 - nb_02_rag_exploration - WARNING - Impossible de déterminer un target_arxiv_id à partir des résultats précédents pour le filtre. Utilisation d'une valeur par défaut ou test désactivé.
2025-06-03 23:31:52 - nb_02_rag_exploration - WARNING - Aucun target_arxiv_id n'a été défini pour le filtre. La recherche filtrée par ArXiv ID est sautée.
2025-06-03 23:31:52 - nb_02_rag_exploration - INFO - --- Fin Stratégie 2 ---


### Stratégie 3: Recherche Hybride (Optionnel)

La recherche hybride combine la recherche vectorielle sémantique avec la recherche par mots-clés traditionnelle (souvent BM25 ou via des index textuels MongoDB). Cela peut être utile pour capturer à la fois la pertinence sémantique et les correspondances exactes de termes.

**Implémentation dans `RetrievalEngine`:**
*   Pour supporter la recherche hybride, le `RetrievalEngine` devrait être capable d'exécuter les deux types de recherche et de fusionner intelligemment les résultats (par exemple, en utilisant Reciprocal Rank Fusion - RRF).
*   Cela nécessiterait que `RetrievalEngine` ait accès non seulement à l'index vectoriel (`VECTOR_INDEX_NAME_FOR_RAG_TEST`) mais aussi à l'index textuel (`TEXT_INDEX_NAME_FOR_RAG_TEST`) créé dans MongoDB.
*   La classe `RetrievalEngine` devrait exposer une méthode dédiée, par exemple `retrieve_hybrid_search(...)`.

**Note pour ce Notebook:**
L'implémentation actuelle de `RetrievalEngine` (selon `src/rag/retrieval_engine.py` au moment de la rédaction de ce notebook) se concentre sur la recherche vectorielle via LlamaIndex et `MongoDBAtlasVectorSearch`. Une véritable recherche hybride nécessiterait des modifications plus substantielles dans `RetrievalEngine` pour intégrer une logique de fusion ou utiliser des fonctionnalités avancées de LlamaIndex pour des retrievers composites.

**Si la recherche hybride n'est pas encore implémentée dans votre `RetrievalEngine`, cette section ne sera pas exécutable.** Vous pouvez la considérer comme une piste d'amélioration future.

Si vous avez implémenté une méthode pour la recherche hybride (ex: `retrieve_hybrid`), vous pouvez décommenter et adapter la cellule de code suivante.

In [7]:
logger.info(f"\n--- Début Stratégie 3: Recherche Hybride (Conceptuel/Optionnel) ---")

# Définir une requête pour le test de recherche hybride
query_for_hybrid_search = selected_test_query 
# query_for_hybrid_search = "Explainable AI for robot vision using Transformers" # Une requête avec des termes spécifiques

logger.info(f"Requête pour la recherche hybride (conceptuelle) : \"{query_for_hybrid_search}\"")

# La cellule suivante est commentée car la recherche hybride nécessite une implémentation spécifique 
# dans RetrievalEngine qui n'est peut-être pas encore présente.

# if retrieval_engine_instance:
#     # Vérifier si la méthode pour la recherche hybride existe
#     if hasattr(retrieval_engine_instance, 'retrieve_hybrid_search'):
#         logger.info("Tentative de recherche hybride...")
#         try:
#             retrieved_nodes_hybrid: Optional[List[RetrievedNode]] = retrieval_engine_instance.retrieve_hybrid_search(
#                 query_text=query_for_hybrid_search,
#                 top_k=5, # La recherche hybride peut bénéficier de plus de candidats initiaux avant fusion
#                 # D'autres paramètres spécifiques à la recherche hybride pourraient être nécessaires:
#                 # alpha=0.5, # Par exemple, un poids pour la fusion RRF
#                 # text_search_weight=0.3, vector_search_weight=0.7
#             )
#             
#             display_retrieved_nodes(
#                 retrieved_nodes_hybrid, 
#                 query_for_hybrid_search, 
#                 search_type="Recherche Hybride"
#             )
# 
#         except Exception as e:
#             logger.error(f"Erreur lors de la recherche hybride : {e}", exc_info=True)
#             print(f"Une erreur est survenue lors de la recherche hybride.")
#     else:
#         logger.warning("La méthode 'retrieve_hybrid_search' n'est pas implémentée dans RetrievalEngine.")
#         print("Fonctionnalité de recherche hybride non disponible dans cette version du RetrievalEngine.")
# else:
#     logger.error("RetrievalEngine non initialisé. Impossible d'exécuter la recherche hybride.")

logger.info("Section de recherche hybride (conceptuelle) passée. Décommentez et adaptez si la fonctionnalité est disponible.")
logger.info(f"--- Fin Stratégie 3 ---")

2025-06-03 23:32:44 - nb_02_rag_exploration - INFO - 
--- Début Stratégie 3: Recherche Hybride (Conceptuel/Optionnel) ---
2025-06-03 23:32:44 - nb_02_rag_exploration - INFO - Requête pour la recherche hybride (conceptuelle) : "What are the main challenges in applying reinforcement learning to robotic manipulation?"
2025-06-03 23:32:44 - nb_02_rag_exploration - INFO - Section de recherche hybride (conceptuelle) passée. Décommentez et adaptez si la fonctionnalité est disponible.
2025-06-03 23:32:44 - nb_02_rag_exploration - INFO - --- Fin Stratégie 3 ---


## Conclusion du Notebook 02: Exploration des Stratégies RAG

Ce notebook a démontré comment utiliser le `RetrievalEngine` pour :
1.  Effectuer une **recherche vectorielle simple** basée sur la similarité sémantique.
2.  Affiner les résultats de la recherche vectorielle en appliquant des **filtres sur les métadonnées** des chunks.
3.  Esquissé le concept de **recherche hybride** comme une amélioration potentielle.

**Points Clés et Pistes d'Amélioration :**

*   **Qualité des Embeddings** : La pertinence des résultats dépend fortement de la qualité des embeddings générés (à la fois pour les chunks stockés et pour la requête). Le choix du modèle d'embedding est crucial.
*   **Stratégie de Chunking** : La manière dont les documents sont segmentés en chunks (taille, chevauchement, respect des frontières sémantiques) impacte directement ce qui peut être récupéré.
*   **Pertinence des Métadonnées** : Des métadonnées riches et précises permettent un filtrage plus efficace et un meilleur contexte pour le LLM en aval.
*   **Techniques de Re-ranking** : Pour améliorer davantage la pertinence, des modèles de re-ranking (par exemple, Cohere Rerank, ou des cross-encoders) peuvent être appliqués sur les résultats initiaux du retriever.
*   **Expansion de Requête** : Des techniques comme HyDE (Hypothetical Document Embeddings) ou la réécriture de requête par un LLM peuvent améliorer la formulation de la requête avant la recherche vectorielle.
*   **Interface Utilisateur** : Les fonctionnalités explorées ici sont les briques de base pour une interface de type question-réponse ou un système de recherche sémantique plus complet.

**Prochaines Étapes Suggérées :**
*   Expérimenter avec différentes requêtes et filtres pour bien comprendre le comportement de votre `RetrievalEngine`.
*   Si la recherche hybride n'est pas implémentée, considérer son ajout pour des cas d'usage nécessitant des correspondances exactes de termes.
*   Passer au **Notebook 03: Développement d'Agents et Outillage**, où nous commencerons à construire des agents intelligents qui pourront utiliser ce moteur de récupération comme un outil.